In [338]:
import pandas as pd
import json
import requests
import pymongo

In [339]:
data = "data/nn_vote_data.csv"

In [340]:
url = "http://api.propublica.org/campaign-finance/v1/"
api_key = {'X-API-KEY': 'X6Z05qcRCfikqOBWSuUQlzT6no3D5EK7bXiV2KkL'}

In [358]:
nn_data = pd.read_csv(data, thousands=',')

In [342]:
nn_vote_df = pd.DataFrame(nn_data)

In [343]:
nn_vote_df.Total = nn_vote_df.Total.astype(float)

ValueError: could not convert string to float: '$794,881 '

In [344]:
head = {'X-API-KEY': 'X6Z05qcRCfikqOBWSuUQlzT6no3D5EK7bXiV2KkL'}
data_fields = {'format': 'json', 'cycle':'2018'}
response = requests.get(url, headers=head)
print(response)

<Response [403]>


In [345]:
response = requests.get(url, headers=head).text
campaign_data_json = json.loads(response)
# api_to_df = pd.DataFrame(campaign_data_json['name'])
print(campaign_data_json)

{'message': 'Missing Authentication Token'}


In [346]:
nn_vote_df.head()

,Name,Chamber,Party,District,Total,Dems,Repubs,Comcast Corp,AT&T Inc,Verizon Communications,March 2017 Vote on Internet Privacy
0,"McCain, John",Senate,R,AZS1,"$794,881",$0,"$794,881","$121,902","$477,088","$195,891",Yea
1,"Hoyer, Steny H",House,D,MD05,"$659,490","$659,490",$0,"$222,100","$249,150","$188,240",Nay
2,"Upton, Fred",House,R,MI06,"$631,000",$0,"$631,000","$222,200","$228,600","$180,200",Yea
3,"Blunt, Roy",Senate,R,MOS1,"$503,133",$0,"$503,133","$152,100","$192,675","$158,358",Yea
4,"Barton, Joe",House,R,TX06,"$495,309",$0,"$495,309","$131,000","$233,509","$130,800",Yea


In [347]:
party_df = nn_vote_df[['Party', 'Total', 'March 2017 Vote on Internet Privacy']]

In [348]:
party_df.head()

,Party,Total,March 2017 Vote on Internet Privacy
0,R,"$794,881",Yea
1,D,"$659,490",Nay
2,R,"$631,000",Yea
3,R,"$503,133",Yea
4,R,"$495,309",Yea


In [356]:
party_df['Total'] = party_df['Total'].str[1:].astype(float)

KeyError: 'Total'

In [350]:
party_df = nn_vote_df.groupby('Party')['Total'].sum()

In [351]:
party_df.head()

Party
D    $659,490 $469,727 $433,500 $385,450 $331,426 $...
I                                    $190,443 $19,000 
R    $794,881 $631,000 $503,133 $495,309 $461,277 $...
Name: Total, dtype: object

In [352]:
vote_df = nn_vote_df[['Party', 'March 2017 Vote on Internet Privacy']]

In [353]:
vote_df = vote_df.groupby('March 2017 Vote on Internet Privacy')['Party'].count()

In [354]:
vote_df.head()

March 2017 Vote on Internet Privacy
Nay           254
Not Voting     17
Yea           264
Name: Party, dtype: int64

In [355]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collection
db_ETL = client.ETL_db
summary_collection = db_ETL.summary